<a href="https://colab.research.google.com/github/Phimphika113/sobeam_year4/blob/main/nurse_w_weekly_workforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyomo ## ติดตั้ง Pyomo
!apt-get install -y glpk-utils  ##Solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
G

In [24]:
from pyomo.environ import *
from pyomo import environ as pe
from pyomo.opt import SolverFactory

# Define model parameters

In [3]:
# Define days
days = range(1, 31) # เซตวันแต่ละเดือน 31 วัน

# Enter shifts of each day
shifts_s = ['morning', 'evening', 'night']  # เซตผลัด
days_shifts_k = {day: shifts_s for day in days}  #ดัชนีผลัด

# Enter nurse ids (name, number, ...)
nurse_N = range(1, 18)  # เซตพยาบาล 17 คน

# Age  of nurse
age_nurse_R = [0] * 17  # เริ่มต้นด้วย 0
age_nurse_R[0] = 2 # ตั้งค่าตัวที่ 1 และ 2 เป็น 2
age_nurse_R[1] = 2
for i in range(2, 17): # ตั้งค่าตัวที่ 3 ถึง 17 เป็น 1
    age_nurse_R[i] = 1

# Age  of nurse
#import numpy as np #เซตอายุของพยาบาล
#age_nurse_R = np.ones(17) #สร้างอาเรย์ยาว 17 และค่าเริ่มต้นเป็น 1
#age_nurse_R[0] = 2 #ค่าตัวที่ 1 และ 2 เป็น 2
#age_nurse_R[1] = 2

# ข้อมูลการมีโรคประจำตัว (มีแค่พยบ 1 คน แล้วตัดออกแล้ว)
#Disease_nurse = np.zeros(17) #สร้างอาเรย์ยาว 17 และค่าเริ่มต้นเป็น 0
#Disease_nurse[0] = 1 #ค่าตัวที่ 1 และ 2 เป็น 2
#Disease_nurse[1] = 1

#กำหนดอายุงานพยาบาล
# เซตอายุงานพยาบาล
work_experience_L = {1: 18,
                              2: 18,
                              3: 15,
                              4: 14,
                              5: 14,
                              6: 12,
                              7: 15,
                              8: 10,
                              9: 9,
                              10: 12,
                              11: 10,
                              12: 9,
                              13: 8,
                              14: 6,
                              15: 1,
                              16: 1,
                              17: 1}

In [4]:
# Create a DataFrame to store nurse information
import pandas as pd
df_nurse = pd.DataFrame ({'Nurse_ID': nurse_N,
                          'Age': age_nurse_R,
                          'Work_Experience': [work_experience_L[i+1] for i in range(17)]})

# Display the DataFrame
print(df_nurse.to_string(index=False))

 Nurse_ID  Age  Work_Experience
        1    2               18
        2    2               18
        3    1               15
        4    1               14
        5    1               14
        6    1               12
        7    1               15
        8    1               10
        9    1                9
       10    1               12
       11    1               10
       12    1                9
       13    1                8
       14    1                6
       15    1                1
       16    1                1
       17    1                1


#Initialize model and variables


In [5]:
# Initialize model
model = ConcreteModel()

# binary variables แสดงการได้รับตารางงานของพยาบาล (Normaltime)
model.nurse_work_X = Var(((i, j, k) for i in nurse_N for j in days for k in days_shifts_k[j]),
                  within=Binary, initialize=0)
# model.nurse_work_Z = Var(((nurse, day, shift) for nurse in nurse_N for day in days for shift in days_shifts_k[day]),
#                 within=Binary, initialize=0)  # ตัวแปรการทำงานในผลัดล่วงเวลา


#Define the objective function

In [6]:
# Define an objective function with model as input, to pass later
def obj_fun(m):
  model.NT = sum(model.nurse_work_X[i, j, k] for i in nurse_N for j in days for k in days_shifts_k[j])  # ผลัดการทำงานทั้งหมดของพยาบาล i ในรอบ 1 เดือน
  alpha = model.NT / len(nurse_N)  # avg of NT, use model.NT here
  differ_NT = model.NT- alpha
  return sum(differ_NT**2 for i in nurse_N) # รวมค่ากำลังสองของความต่างสำหรับพยาบาลแต่ละคน

# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_fun, sense=minimize)

#Model constraints

In [7]:
# 1.ช บ ด ผลัดละ 5 คน
model.const1 = ConstraintList()
for j in days:
    for k in days_shifts_k[j]:
            model.const1.add(
                5 == sum(model.nurse_work_X[i, j, k]  for j in days for k in days_shifts_k[j])
            )

In [ ]:
#พยาบาลแต่ละคน ทำงานไม่เกิน 40 ชม. ต่อสัปดาห์ หรือ 5 เวรต่อสัปดาห์
#ไม่ได้ใช้ g คู่กับ j ได้มั้ย แต่ day งานตัวอย่างมี7อยู่แล้ว
model.const2 = ConstraintList()
for i in nurse_N:
    model.constraints.add(
        5 >= sum(model.nurse_work_X[i, j, k] for j in days for k in days_shifts_k[j])
    )


#Solver

In [20]:
#!cbc --version

Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - cbc --version (default strategy 1)
No match for -version - ? for list of commands
CoinSolver takes input from arguments ( - switches to stdin)
Enter ? for list of commands or help
Coin:^C


In [25]:
!sudo apt-get install  glpk-utils
solver = pe.SolverFactory('glpk', executable='/user/bin/glpsol')
from pyomo.opt import SolverFactory
opt = SolverFactory('glpk')
opt.solve(model, tee=True)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/base/solvers.py", line 148, in __call__
    opt = self._cls[_name](**kwds)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/solvers/plugins/solvers/GLPK.py", line 92, in __init__
    SystemCallSolver.__init__(self, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 66, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 115, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver glpk. File with name=/user/bin/glpsol either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.


ValueError: Model objective (obj) contains nonlinear terms that cannot be written to LP format

In [ ]:
from pyomo.environ import *

# Specify the path to the CBC executable
solver = SolverFactory('cbc', executable='/path/to/cbc')
results = solver.solve(model)

In [ ]:
#import pandas as pd
#from pyomo.environ import ConcreteModel, Var, Param, Constraint, Binary, NonNegativeIntegers, Objective, SolverFactory, NonNegativeReals

# ... (โค้ดเดิมของคุณสำหรับการสร้างโมเดล)

# แก้ไขโมเดลโดยใช้ solver
solver = SolverFactory('glpk')  # หรือ solver อื่นๆ ที่คุณต้องการใช้
results = solver.solve(model)

# สร้าง DataFrame ว่างสำหรับเก็บตารางการทำงาน
schedule = pd.DataFrame(index=nurses_N, columns=days)

# กำหนดชื่อของผลัด
#shifts = {1: "เช้า", 2: "บ่าย", 3: "ดึก"}

# เติมข้อมูลลงในตาราง
for i in nurse_N:
    for j in days:
        for k in days_shifts_k:
            if model.X[i, j, k].value == 1:  # ถ้าพยาบาลทำงานในผลัดนั้น
                schedule.at[i, j] = days_shifts_s[k]

# แสดงตารางเวลา
print(schedule)


# ติดไว้ก่อน
- คิดแค่NTไม่ได้เอาOTมาคิด (ทั้งตัวแปรตัดสินใจ และ ใน obj.fun
- ตัดโรคประจะตัวออก (พยาบาล จะมี attribute : id, อายุพยาบาล, อายุงาน)